In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [5]:
ruta_base = "../Asistencia/Data/Base.xlsx"

In [6]:
df = pd.read_excel(ruta_base)

In [7]:
print(f"Dimensiones: {df.shape}")
print(f"\nColumnas: {df.columns.tolist()}")
print(f"\nTipos de datos:")
print(df.dtypes)
print(f"\nPrimeras filas:")
print(df.head())

Dimensiones: (720471, 17)

Columnas: ['MovieID', 'Title', 'Genre', 'ReleaseYear', 'ReleaseDate', 'Country', 'BudgetUSD', 'US_BoxOfficeUSD', 'Global_BoxOfficeUSD', 'Opening_Day_SalesUSD', 'One_Week_SalesUSD', 'IMDbRating', 'RottenTomatoesScore', 'NumVotesIMDb', 'NumVotesRT', 'Director', 'LeadActor']

Tipos de datos:
MovieID                          int64
Title                           object
Genre                           object
ReleaseYear                      int64
ReleaseDate             datetime64[ns]
Country                         object
BudgetUSD                        int64
US_BoxOfficeUSD                  int64
Global_BoxOfficeUSD              int64
Opening_Day_SalesUSD             int64
One_Week_SalesUSD                int64
IMDbRating                       int64
RottenTomatoesScore              int64
NumVotesIMDb                     int64
NumVotesRT                       int64
Director                        object
LeadActor                       object
dtype: object

Prime

In [8]:
print("Valores Nulos")
null_analysis = pd.DataFrame({
    'Columna': df.columns,
    'Valores_Nulos': df.isnull().sum(),
    'Porcentaje_Nulos': (df.isnull().sum() / len(df)) * 100,
    'Valores_Blancos': (df == '').sum(),
    'Valores_0': (df == 0).sum()
})
print(null_analysis)

Valores Nulos
                                   Columna  Valores_Nulos  Porcentaje_Nulos  \
MovieID                            MovieID              0               0.0   
Title                                Title              0               0.0   
Genre                                Genre              0               0.0   
ReleaseYear                    ReleaseYear              0               0.0   
ReleaseDate                    ReleaseDate              0               0.0   
Country                            Country              0               0.0   
BudgetUSD                        BudgetUSD              0               0.0   
US_BoxOfficeUSD            US_BoxOfficeUSD              0               0.0   
Global_BoxOfficeUSD    Global_BoxOfficeUSD              0               0.0   
Opening_Day_SalesUSD  Opening_Day_SalesUSD              0               0.0   
One_Week_SalesUSD        One_Week_SalesUSD              0               0.0   
IMDbRating                      IMDbRa

In [10]:
def limpiar_datos(df):
    df_clean = df.copy()
    
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    
    columnas_clave = ['presupuesto', 'ingresos', 'ganancias', 'rentabilidad']
    
    for col in columnas_clave:
        if col in df_clean.columns:
            if col in ['presupuesto', 'ingresos']:
                df_clean = df_clean[df_clean[col] > 0]
    
    text_cols = ['titulo', 'director', 'genero', 'actores']
    for col in text_cols:
        if col in df_clean.columns:
            # Eliminar espacios en blanco
            df_clean[col] = df_clean[col].str.strip()
            # Reemplazar valores vacíos
            df_clean[col] = df_clean[col].replace('', 'Desconocido')
    
    date_cols = ['fecha_estreno', 'año']
    for col in date_cols:
        if col in df_clean.columns:
            # Convertir a formato fecha
            try:
                df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
            except:
                pass
    
    #Calcular rentabilidad si no existe
    if 'presupuesto' in df_clean.columns and 'ingresos' in df_clean.columns:
        df_clean['rentabilidad'] = (df_clean['ingresos'] - df_clean['presupuesto']) / df_clean['presupuesto']
    
    return df_clean

In [ ]:
df_clean = limpiar_datos(df)
print(f"Filas despues de limpieza: {df_clean.shape}")

Filas después de limpieza: (720471, 17)


In [ ]:
def analisis_estadistico_mejorado(df):
    print("=== ANÁLISIS ESTADISTICO MEJORADO ===\n")
    
    # 1. IDENTIFICAR Y FILTRAR VALORES IRREALES
    print("1. FILTRADO DE VALORES IRREALES")
    
    limites_realistas = {
        'BudgetUSD': (10000, 500000000),      
        'US BoxOfficeUSD': (1000, 1000000000), 
        'Global BoxOfficeUSD': (1000, 3000000000) 
    }
    
    df_filtrado = df.copy()
    filas_antes = len(df_filtrado)
    
    for columna, (min_real, max_real) in limites_realistas.items():
        if columna in df_filtrado.columns:
            # Contar valores irreales
            irreales = ((df_filtrado[columna] < min_real) | (df_filtrado[columna] > max_real)).sum()
            print(f"   {columna}: {irreales} valores fuera de rango realista (${min_real:,} - ${max_real:,})")
            
            # Filtrar
            df_filtrado = df_filtrado[
                (df_filtrado[columna] >= min_real) & 
                (df_filtrado[columna] <= max_real)
            ]
    
    print(f"   Filas eliminadas por valores irreales: {filas_antes - len(df_filtrado)}")
    print(f"   Filas restantes: {len(df_filtrado)}\n")
    
    # 2. ESTADÍSTICAS ROBUSTAS (resistentes a outliers)
    columnas_numericas = ['BudgetUSD', 'US BoxOfficeUSD', 'Global BoxOfficeUSD', 'IMDbRating']
    
    for col in columnas_numericas:
        if col in df_filtrado.columns:
            print(f"2. ANÁLISIS DE {col}")
            datos = df_filtrado[col].dropna()
            
            # Estadísticas básicas
            stats = {
                'Count': len(datos),
                'Mean': f"${datos.mean():,.0f}" if 'USD' in col else f"{datos.mean():.2f}",
                'Median': f"${datos.median():,.0f}" if 'USD' in col else f"{datos.median():.2f}",
                'Std': f"${datos.std():,.0f}" if 'USD' in col else f"{datos.std():.2f}",
                'Min': f"${datos.min():,.0f}" if 'USD' in col else f"{datos.min():.2f}",
                'Max': f"${datos.max():,.0f}" if 'USD' in col else f"{datos.max():.2f}",
            }
            
            for key, value in stats.items():
                print(f"   {key}: {value}")
            
            # Detección de outliers usando IQR (método más robusto)
            Q1 = datos.quantile(0.25)
            Q3 = datos.quantile(0.75)
            IQR = Q3 - Q1
            limite_inferior = Q1 - 1.5 * IQR
            limite_superior = Q3 + 1.5 * IQR
            
            outliers = datos[(datos < limite_inferior) | (datos > limite_superior)]
            print(f"   Outliers detectados (IQR): {len(outliers)} ({len(outliers)/len(datos)*100:.1f}%)\n")
    
    return df_filtrado

# Aplicar análisis mejorado
df_mejorado = analisis_estadistico_mejorado(df)

=== ANÁLISIS ESTADÍSTICO MEJORADO ===

1. FILTRADO DE VALORES IRREALES
   BudgetUSD: 252725 valores fuera de rango realista ($10,000 - $500,000,000)
   Filas eliminadas por valores irreales: 252725
   Filas restantes: 467746

2. ANÁLISIS DE BudgetUSD
   Count: 467746
   Mean: $165,742,524
   Median: $129,001,121
   Std: $135,202,088
   Min: $100,000
   Max: $499,995,271
   Outliers detectados (IQR): 0 (0.0%)

2. ANÁLISIS DE IMDbRating
   Count: 467746
   Mean: 58.33
   Median: 63.00
   Std: 22.63
   Min: 1.00
   Max: 99.00
   Outliers detectados (IQR): 50880 (10.9%)



In [ ]:
def comparar_estadisticas(df_original, df_filtrado):

    print("COMPARATIVA: ESTADISTICAS ORIGINALES vs FILTRADAS\n")
    
    columnas_comparar = ['BudgetUSD', 'US BoxOfficeUSD', 'Global BoxOfficeUSD']
    
    for col in columnas_comparar:
        if col in df_original.columns:
            print(f"--- {col} ---")
            
            # Original (con outliers)
            orig_data = df_original[col].dropna()
            # Filtrado (sin valores irreales)
            filt_data = df_filtrado[col].dropna()
            
            comparacion = pd.DataFrame({
                'Métrica': ['Count', 'Mean', 'Median', 'Std', 'Min', 'Max'],
                'Original': [
                    len(orig_data),
                    f"${orig_data.mean():,.0f}",
                    f"${orig_data.median():,.0f}", 
                    f"${orig_data.std():,.0f}",
                    f"${orig_data.min():,.0f}",
                    f"${orig_data.max():,.0f}"
                ],
                'Filtrado': [
                    len(filt_data),
                    f"${filt_data.mean():,.0f}",
                    f"${filt_data.median():,.0f}",
                    f"${filt_data.std():,.0f}",
                    f"${filt_data.min():,.0f}",
                    f"${filt_data.max():,.0f}"
                ],
                'Mejora': [
                    f"{(len(filt_data)/len(orig_data)*100):.1f}%",
                    f"{(filt_data.mean()/orig_data.mean()*100 if orig_data.mean() > 0 else 0):.1f}%",
                    f"{(filt_data.median()/orig_data.median()*100 if orig_data.median() > 0 else 0):.1f}%",
                    f"{(filt_data.std()/orig_data.std()*100):.1f}%",
                    "-",
                    "-"
                ]
            })
            
            print(comparacion.to_string(index=False))
            print()

# Ejecutar comparación
comparar_estadisticas(df, df_mejorado)

=== COMPARATIVA: ESTADÍSTICAS ORIGINALES vs FILTRADAS ===

--- BudgetUSD ---
Métrica       Original     Filtrado Mejora
  Count         720471       467746  64.9%
   Mean   $731,244,797 $165,742,524  22.7%
 Median   $272,352,745 $129,001,121  47.4%
    Std $1,238,772,101 $135,202,088  10.9%
    Min       $100,000     $100,000      -
    Max $9,999,646,368 $499,995,271      -



In [ ]:
def analizar_rentabilidad_corregido(df_filtrado):
    
    # 1. VERIFICAR QUE TENEMOS LAS COLUMNAS NECESARIAS
    print("1. VERIFICACION DE COLUMNAS DISPONIBLES:")
    columnas_disponibles = df_filtrado.columns.tolist()
    print(f"   Columnas en el dataset: {columnas_disponibles}")
    
    columnas_presupuesto = [col for col in columnas_disponibles if 'budget' in col.lower() or 'presupuesto' in col.lower()]
    columnas_ingresos = [col for col in columnas_disponibles if 'boxoffice' in col.lower() or 'ingresos' in col.lower() or 'revenue' in col.lower()]
    
    print(f"   Columnas de presupuesto encontradas: {columnas_presupuesto}")
    print(f"   Columnas de ingresos encontradas: {columnas_ingresos}")
    
    if not columnas_presupuesto or not columnas_ingresos:
        print(" ERROR: No se encontraron las columnas")
        return df_filtrado
    
    # Usar las primeras columnas encontradas
    col_presupuesto = columnas_presupuesto[0]
    col_ingresos = columnas_ingresos[0]
    
    print(f" Columnas: '{col_presupuesto}' y '{col_ingresos}'\n")
    
    print("2. VERIFICACION DE CALIDAD DE DATOS:")
    print(f"   Filas totales: {len(df_filtrado)}")
    print(f"   Valores nulos en {col_presupuesto}: {df_filtrado[col_presupuesto].isnull().sum()}")
    print(f"   Valores nulos en {col_ingresos}: {df_filtrado[col_ingresos].isnull().sum()}")
    print(f"   Valores cero en {col_presupuesto}: {(df_filtrado[col_presupuesto] == 0).sum()}")
    
    df_calculo = df_filtrado[
        (df_filtrado[col_presupuesto].notnull()) & 
        (df_filtrado[col_ingresos].notnull()) &
        (df_filtrado[col_presupuesto] > 0) 
    ].copy()
    
    print(f"   Filas validas: {len(df_calculo)}\n")
    
    if len(df_calculo) == 0:
        print(" ERROR: No hay datos validos para calcular rentabilidad")
        return df_filtrado
    
    # 4. CALCULAR RENTABILIDAD
    print("3. CALCULO DE RENTABILIDAD:")
    df_calculo['Profitability'] = (df_calculo[col_ingresos] - df_calculo[col_presupuesto]) / df_calculo[col_presupuesto]
    
    # 5. ESTADÍSTICAS DETALLADAS
    print("4. ESTADISTICAS DE RENTABILIDAD:")
    print(f"   • Peliculas rentables: {(df_calculo['Profitability'] > 0).sum()} ({(df_calculo['Profitability'] > 0).mean()*100:.1f}%)")
    print(f"   • Peliculas con perdidas: {(df_calculo['Profitability'] < 0).sum()} ({(df_calculo['Profitability'] < 0).mean()*100:.1f}%)")
    print(f"   • Peliculas en equilibrio: {(df_calculo['Profitability'] == 0).sum()} ({(df_calculo['Profitability'] == 0).mean()*100:.1f}%)")
    print(f"   • Rentabilidad promedio: {df_calculo['Profitability'].mean()*100:+.1f}%")
    print(f"   • Mediana de rentabilidad: {df_calculo['Profitability'].median()*100:+.1f}%")
    print(f"   • MAxima rentabilidad: {df_calculo['Profitability'].max()*100:+.1f}%")
    print(f"   • MInima rentabilidad: {df_calculo['Profitability'].min()*100:+.1f}%")
    print(f"   • Desviación estándar: {df_calculo['Profitability'].std()*100:.1f}%")
    
    # 6. ANÁLISIS POR QUINTILES
    print("\n5. ANALISIS POR QUINTILES DE RENTABILIDAD:")
    quintiles = df_calculo['Profitability'].quantile([0.2, 0.4, 0.6, 0.8])
    
    print("   Quintiles (puntos de corte):")
    for q, value in quintiles.items():
        print(f"   • {int(q*100)} percentil: {value*100:+.1f}%")
    
    # 7. CATEGORIZACIÓN DETALLADA
    print("\n6. CATEGORIZACION DETALLADA:")
    condiciones = [
        df_calculo['Profitability'] < -0.5,    
        df_calculo['Profitability'] < 0,       
        df_calculo['Profitability'] == 0,      
        df_calculo['Profitability'] < 0.5,     
        df_calculo['Profitability'] < 1,        
        df_calculo['Profitability'] < 3,        
        df_calculo['Profitability'] >= 3       
    ]
    
    categorias = [
        'Pérdida catastrofica (<-50%)',
        'Pérdida moderada (-50%-0%)', 
        'Equilibrio exacto (0%)',
        'Rentabilidad baja (0-50%)',
        'Rentabilidad media (50-100%)',
        'Rentabilidad alta (100-300%)',
        'Exito alto (>300%)'
    ]
    
    df_calculo['ProfitCategory'] = np.select(condiciones, categorias, default='Fuera de rango')
    
    distribucion = df_calculo['ProfitCategory'].value_counts()
    for categoria, count in distribucion.items():
        porcentaje = (count / len(df_calculo)) * 100
        print(f"   • {categoria}: {count} peliculas ({porcentaje:.1f}%)")
    
    # 8. EJEMPLOS DE CADA CATEGORIA
    print("\n7. EJEMPLOS POR CATEGORIA (primera pelIcula de cada una):")
    for categoria in categorias:
        ejemplo = df_calculo[df_calculo['ProfitCategory'] == categoria]
        if len(ejemplo) > 0:
            primera_peli = ejemplo.iloc[0]
            # Buscar columna de título
            col_titulo = [col for col in df_calculo.columns if 'title' in col.lower() or 'titulo' in col.lower() or 'name' in col.lower()]
            nombre_peli = primera_peli[col_titulo[0]] if col_titulo else "Sin nombre"
            
            print(f"   • {categoria}: {nombre_peli} (Presupuesto: ${primera_peli[col_presupuesto]:,.0f}, Rentabilidad: {primera_peli['Profitability']*100:+.1f}%)")
    
    df_filtrado['Profitability'] = np.nan
    df_filtrado['ProfitCategory'] = 'No calculado'
    df_filtrado.update(df_calculo[['Profitability', 'ProfitCategory']])
    
    return df_filtrado

df_final_corregido = analizar_rentabilidad_corregido(df_mejorado)

🚀 EJECUTANDO ANÁLISIS DE RENTABILIDAD CORREGIDO...
1. VERIFICACION DE COLUMNAS DISPONIBLES:
   Columnas en el dataset: ['MovieID', 'Title', 'Genre', 'ReleaseYear', 'ReleaseDate', 'Country', 'BudgetUSD', 'US_BoxOfficeUSD', 'Global_BoxOfficeUSD', 'Opening_Day_SalesUSD', 'One_Week_SalesUSD', 'IMDbRating', 'RottenTomatoesScore', 'NumVotesIMDb', 'NumVotesRT', 'Director', 'LeadActor', 'Profitability', 'ProfitCategory']
   Columnas de presupuesto encontradas: ['BudgetUSD']
   Columnas de ingresos encontradas: ['US_BoxOfficeUSD', 'Global_BoxOfficeUSD']
 Columnas: 'BudgetUSD' y 'US_BoxOfficeUSD'

2. VERIFICACIÓN DE CALIDAD DE DATOS:
   Filas totales: 467746
   Valores nulos en BudgetUSD: 0
   Valores nulos en US_BoxOfficeUSD: 0
   Valores cero en BudgetUSD: 0
   Filas validas: 467746

3. CALCULO DE RENTABILIDAD:
4. ESTADÍSTICAS DE RENTABILIDAD:
   • Películas rentables: 320131 (68.4%)
   • Películas con perdidas: 147615 (31.6%)
   • Películas en equilibrio: 0 (0.0%)
   • Rentabilidad promedio: 